In [ ]:
import logging
from datetime import date
from enum import Enum
import requests
from bs4 import BeautifulSoup

from fastapi import APIRouter,Depends
from fastapi_utils.cbv import cbv
from pydantic import BaseModel
from sqlalchemy import func, select, case, delete, insert
from sqlalchemy.ext.asyncio import AsyncSession


from common.db import db_etc
from models.models_etc import KoreabankIndexYearly


router = APIRouter()

logger = logging.getLogger('uvicorn')



@cbv(router=router)

class koreagdp:
    db:AsyncSession = Depends(db_etc.get_db)


    @router.get('/koreagdpearly',
        summary='년간 한국 gdp',
    )
    async def gdpkorea(self):

        url = 'https://www.index.go.kr/unity/potal/eNara/sub/showStblGams3.do?stts_cd=273601&idx_cd=2736&freq=Y&period=1970:2021'
        
        header = { 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
            'accept-language' : 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
            'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
            
        data = {'stts_cd' : '273601', 'idx_cd': '2736' ,'freq': 'Y', 'period':'1970:2021'}
        
        res = requests.get(url,headers=header , data =data)
        
        soup = BeautifulSoup(res.text ,'html.parser')
        
        
        tbody = soup.find('tbody') # 명목 GDP, 실질 GDP성장률 수집때 필요 
        


         # 'year'
         # 년도 항목 가져오기
        thead = soup.find('thead')
        
        
        ths = thead.find_all('th' ,{'class':'tc'})

        resultyear=[]
        
        for th in ths:
            
            itemyear =  f'{th.text}'
            
            resultyear.append(itemyear)



        #  'Nominal GDP' 
        # # 국내 총 생산 (명목 GDP) 열 정보

        tr1 = tbody.find_all('tr' , {'id' : 'tr_273601_1'} )
        
        resultnominal =[]
        
        for tr in tr1:
            
            tds = tr.find_all('td' )
            
            for td in tds:
                
                itemnominal = f'{td.text}'
                
                
                
                resultnominal.append(itemnominal)


        # 'real GDP growth rate' :
        # 경제성장률 (실질 GDP 성장률) 
        tr2 = tbody.find_all('tr',{'id' : 'tr_273601_2'})
        
        resultreal =[]
        
        for trr in tr2:
            
            tdss = trr.find_all('td' )
            
            for tdd in tdss:
                
                itemreal = f'{tdd.text}'
                
                
                resultreal.append(itemreal) 

        
        #  return resultyear , resultnominal,resultreal



        res =[]
         
        for ryear,tnominal,rreal in zip(resultyear,resultnominal,resultreal):
            item ={
                
                'date': ryear + "-12-31",  #날짜로
                'nominalgdp' :tnominal.replace(',', ''),
                'real_gdp_growthrate': float(rreal),
                }
                
            res.append(item)

                
            # print(res)

        
        return res


        

   

    @router.post('/koreagdpupdate',
        summary= 'koreagdpyearly 업데이트')


    async def updated_gdpkorea(self):

        try:
            data = await self.gdpkorea()
            
            await self.db.execute(
                insert(KoreabankIndexYearly),
                data,
                )
                
            await self.db.commit()

            print("good")

        except Exception:
            await self.db.rollback()

            logger.exception('한국 년간 gdp 정보')

        return None
    # db 저장되어있는 데이터 불러오기
    # @router.get('/originkoreadata')
    async def get_origin_gdpkoreayear(self):

        try:
            q=select(KoreabankIndexYearly)
            rs = await self.db.execute(q)
            return rs.scalars().all()

        except Exception as e:
            print(e)
            
    # 새로 생긴 데이터 db에 추가해주기
    async def koreabankyearnewdata(self,original,new):
        for n in new:
            found=False
            for o in original:
                if n['date'] == str(o.date):
                    found =True
                
                    break

            if not found:
                n1 = KoreabankIndexYearly(**n)
                self.db.add(n1)

                print(n['date']+ " update success")
                
    
    print("finish")

        


    @router.post('/koreabankyearlyupdated',
            summary = 'koreabank 1년주기 업데이트')

    async def koreabankyearlyupdated(self):

        print("start")

        try:
            new = await self.gdpkorea()
            
            original:list[KoreabankIndexYearly] = await self.get_origin_gdpkoreayear()
           

            for o in original:
                
                found = False
            

                for n in new: 
                   
                    if str(o.date) == n['date']:
                     
                 
                        n1=KoreabankIndexYearly(**n) 
                        o.date=n1.date
                        o.nominalgdp=n1.nominalgdp
                        o.real_gdp_growthrate=n1.real_gdp_growthrate
                        found=True
                       
                        break

                if not found:
                      print(str(o.date) + " check please")
                      
                 
                # if i > 0 and i % 10 == 0:
                #     await self.db.commit()

            await self.koreabankyearnewdata(original,new)
            await self.db.commit()

        except Exception as e:
            print(f"error:{e}")

            return None




